In [1]:
from ultralytics import YOLO
 # Можно заменить 'yolov8s-seg.pt' на 'yolov8m-seg.pt' или другую версию
import torch
import numpy as np
import cv2
from scipy.spatial.distance import directed_hausdorff

In [2]:
model = YOLO('s70.pt') 
model.names

{0: 'redOring', 1: 'YellowOring', 2: 'GreyOring'}

In [6]:

import numpy as np
import cv2
from scipy.spatial.distance import directed_hausdorff

# Функция для выполнения операции "closing" на тензоре маски
def closing(mask_tensor, kernel_size=5):
    mask_np = mask_tensor.cpu().numpy().astype(np.uint8)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    closed_mask = cv2.morphologyEx(mask_np, cv2.MORPH_CLOSE, kernel)
    return closed_mask

# Функция для получения контура маски
def get_contour(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        return max(contours, key=cv2.contourArea)
    else:
        return None

# Функция для вычисления угла поворота контура
def get_rotation_angle(contour):
    if contour is not None:
        rect = cv2.minAreaRect(contour)
        angle = rect[-1]
        return angle
    else:
        return None

# Функция для сравнения схожести двух контуров (с помощью метрики Хаусдорфа)
def contour_similarity(contour1, contour2):
    if contour1 is None or contour2 is None:
        return None
    # Преобразуем контуры в один массив точек
    contour1 = np.squeeze(contour1)
    contour2 = np.squeeze(contour2)
    # Вычисляем расстояние Хаусдорфа
    hausdorff_distance = max(directed_hausdorff(contour1, contour2)[0], directed_hausdorff(contour2, contour1)[0])
    return hausdorff_distance

# Главная функция для сравнения масок
def compare_masks(mask1_tensor, mask2_tensor):
    # Выполняем операцию closing для обеих масок
    mask1 = closing(mask1_tensor)
    mask2 = closing(mask2_tensor)

    # Получаем контуры масок
    contour1 = get_contour(mask1)
    contour2 = get_contour(mask2)

    # Вычисляем углы поворота для обоих контуров
    angle1 = get_rotation_angle(contour1)
    angle2 = get_rotation_angle(contour2)

    if angle1 is not None and angle2 is not None:
        angle_difference = abs(angle1 - angle2)
        print(f"Разница в углах: {angle_difference} градусов")
    else:
        print("Не удалось найти контуры для одной из масок")

    # Сравниваем схожесть контуров
    similarity = contour_similarity(contour1, contour2)
    if similarity is not None:
        print(f"Схожесть контуров (метрика Хаусдорфа): {similarity}")
    else:
        print("Не удалось найти контуры для одной из масок")

In [5]:
redOringRotated = "frame_117.jpg"
redOringTemplate = "frame_143.jpg"
rotated_pred = model.predict("./images/"+redOringRotated, save=True,retina_masks=True,show_labels=False, show_conf=False, show_boxes=False)[0]
temp_pred = model.predict("./images/frame_143.jpg", save=True,retina_masks=True,show_labels=False, show_conf=False, show_boxes=False)[0]
rotMask = rotated_pred.masks.data[0]
tempMask = temp_pred.masks.data[0]
compare_masks(rotMask, tempMask)



image 1/1 /media/timofeym/hddb5/programming/Python/DataScience&ML/OpenCodeTest/images/frame_117.jpg: 384x640 1 YellowOring, 1 GreyOring, 12.4ms
Speed: 2.0ms preprocess, 12.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict2

image 1/1 /media/timofeym/hddb5/programming/Python/DataScience&ML/OpenCodeTest/images/frame_143.jpg: 384x640 1 YellowOring, 1 GreyOring, 12.1ms
Speed: 1.7ms preprocess, 12.1ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict2
Разница в углах: 45.0 градусов
Схожесть контуров (метрика Хаусдорфа): 52.354560450833695
